In [2]:
import pandas as pd #reading and writing dataset
import numpy as np
import re #regex , remove unwanted text , select patterns

import matplotlib.pyplot as plt #charts analyze data visualization
from PIL import Image #close read write images

from collections import Counter #Importing Counter to count items in collections
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator #close read write images

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [4]:
!pip install vaderSentiment

In [5]:
def date_time(s):
  pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+)(AM|PM|am|pm)?-' # Adjusted pattern
  result = re.match(pattern, s)
  if result:
    return True
  return False


In [6]:
def find_author(s):
  s = s.split(":")
  if len(s)==2: # If there are two parts after splitting, it’s likely the message has an author
    return True
  else:
    return False

In [7]:
def msgs(line):
  splitline = line.split(' - ')
  dateTime = splitline[0] #The part before " - " is the timestamp
  date, time = dateTime.split(", ")
  message = " ".join(splitline[1:])

  if find_author(message): # Checks if message has an author
    splitmessage = message.split(": ") # Splits message at ": " to separate author from the actual message
    author = splitmessage[0] # The first part is the author’s name
    message = " ".join(splitmessage[1:])   # The rest is the actual message content
  else:
    author= None #If no author, set author to None
  return date, time, author, message # Returns date, time, author, and message as a tuple

In [8]:
data = [] # Initializes an empty list to store message data
conversation = 'Whatsapp_chat.csv'
with open(conversation, encoding="utf-8") as fp:
  fp.readline() # Skips the first line
  messageBuffer = [] # Buffer to hold lines that belong to the same message
  date, time, author = None, None, None

  while True:  # Starts an infinite loop to read lines until the end of the file
    line = fp.readline() # Reads the next line in the file
    if not line:
      break
    line = line.strip() # Removes leading and trailing whitespace from the line
    if date_time(line):
      if len(messageBuffer) > 0:
        data.append([date, time, author, ' '.join(messageBuffer)]) # Adds previous message data to `data` list
      messageBuffer.clear()
      date, time, author, message = msgs(line)
      messageBuffer.append(message)
    else:
      messageBuffer.append(line)

In [ ]:
df = pd.DataFrame(data, columns=["Date", 'Time', 'Author', 'Message'])
df['Date'] = pd.to_datetime(df['Date'])

data = df.dropna()

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sentiments = SentimentIntensityAnalyzer()

data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]

print(data.head())

In [56]:
#validate
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])

def score(a, b, c):
  if (a>b) and (a>c):
    print("Positive 😊 ")
  elif (b>a) and (b>c):
    print("Negative 😠 ")
  else:
    print("Neutral 🙂 ")

In [57]:
score(x,y,z)

Neutral 🙂 
